In [ ]:
!nvidia-smi

Mon Oct 23 14:33:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    17W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.34.1
Uninstalling transformers-4.34.1:
  Successfully uninstalled transformers-4.34.1
Found existing installation: accelerate 0.23.0
Uninstalling accelerate-0.23.0:
  Successfully uninstalled accelerate-0.23.0
  Using cached transformers-4.34.1-py3-none-any.whl (7.7 MB)
  Using cached accelerate-0.23.0-py3-none-any.whl (258 kB)


In [ ]:
from transformers import pipeline,set_seed
from datasets import load_dataset,load_from_disk,load_metric
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
device="cuda" if torch.cuda.is_available() else "cpu"
print(device)
model_ckpt="google/pegasus-cnn_dailymail"

tokenizer=AutoTokenizer.from_pretrained(model_ckpt)

model=AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

cuda


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
!unzip summarizer-data.zip

--2023-10-23 14:35:08--  https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip [following]
--2023-10-23 14:35:08--  https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip.1’

summarizer-data.zip 100%[===================>]   7.54M  --.-KB/s    in 0.1s    

2023-10-23 14:35:09 (77.4 MB/s) - ‘summarizer-data.zip.1’ saved [7903594

In [ ]:
dataset_samsum=load_from_disk("samsum_dataset")
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings=tokenizer(example_batch['dialogue'],max_length=1024,truncation=True)

    with tokenizer.as_target_tokenizer():
        target_encodings=tokenizer(example_batch['summary'],max_length=128,truncation=True)

    return {
        "input_ids":input_encodings['input_ids'],
        "attention_mask":input_encodings['attention_mask'],
        "labels":input_encodings['input_ids']
    }

In [ ]:
dataset_samsum_feat=dataset_samsum.map(convert_examples_to_features,batched=True)

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3848: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
dataset_samsum_feat['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [ ]:
from transformers.data.data_collator import DataCollatorForSeq2Seq
seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer,model=model)

In [ ]:
from transformers import TrainingArguments,Trainer

trainer_args=TrainingArguments(
    output_dir='pegasus-samsum',num_train_epochs=15,warmup_steps=500,
    per_device_train_batch_size=8, per_device_eval_batch_size=8,
    weight_decay=0.01,logging_steps=10,
    evaluation_strategy='steps',eval_steps=500,save_steps=1e6,
    gradient_accumulation_steps=16
)

In [ ]:
trainer=Trainer(model=model,args=trainer_args,
                tokenizer=tokenizer,data_collator=seq2seq_data_collator,
                train_dataset=dataset_samsum_feat['test'],
                eval_dataset=dataset_samsum_feat['validation'])

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: ignored